<a href="https://colab.research.google.com/github/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION/blob/main/Algoritmos_isaac_ropero_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  # AG3 - Actividad Guiada 3
  ## Isaac Ropero Mendez
  ### https://github.com/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION/


He añadido algunas cosas ya que me resultó interesante el algoritmo. Pero sigue manteniendo la esencia de lo que se hizo en clase.